# Multilabel classification

### Ebben a notebookban végzem el, azt hogy miből fogyaszt az adott személy az adott napon.

* A collaborative filtering segítségével sikerült megszereznem egy olyan változót, hogy melyik terméket milyen magasra értékelik a kollégisták
* Ennek a "rating" változónak a segítségével tudok a termékek között személyenként különbséget tenni, anélkül hogy tudnom kéne a napi szintű közelmúltbeli fogyasztásukat a termékből
* A rating és a világban (koliban) történő események számából, abból hogy mennyit fogyaszt az adott ember a napon és hogy a hét melyik napja van, multilabel classificationt végzek el. Ezzel, a top5 termékből megmondom hogy azon a napon melyiket fognak fogyasztani.

### Disclaimerek:

* Az adat mennyisége szar, habár minden termékre megvan a ratingje az embereknek, a termék szűrés mentén felmerülnek olyan problémák, hogy mivel konkrét márkák a termék megfigyeléseim, ezért nagyon szétoszlik a fogyasztás a márkák között, bőven benne van hogy más termékekből fogyasztanak a top fogyasztók. Emiatt áll fel az a helyzet, hogy a top10 fogyasztóból a top5 terméket csak 5-en fogyasztják a kettészelt train és test adaton. Sajnos akkor nem lehet elvégezni a multilabel classificationt, hogyha az egyik emberk nem fogyaszt valamelyik termékből egyet sem. Ezt a továbbdolgozás mentén valahogy muszáj kikerülni, viszont jelenleg nemtudom hogy hogyan.
* Ahhoz hogy ez a modell teljes legyen, szükséges lenne, hogy az adott napi fogyasztás mértékét megbecsüljem, ne pedig egy fix változóként kelljen berakni. 
* Hogyha (a terveim szerint ezt kellett volna még megcsinálom) készen lenne a modell, amely az események száma, események típusa és az adott személy tartozásának (az előző napon, héten) segítségével megbecsüli mennyi lesz a személy adott napi összfogyasztása.
* Hogyha az összfogyasztás abból a modellből kijön, akkor azt ebbe a modellbe beillesztve lehetséges lenne megmondani azt is hogy az összfogyasztás milyen termékek között oszlik meg.

### Notebook struktúrája

* Adat előkészítése, amelyben a változókat olyan formára hozom hogy lehessen labeleket becsülni
* Az alap modell megismerése, annak felépítése, kiértékelése az én személyemen
* Ezekután személyenkénti modellek tanítása és aztán azokkal való predikció, egy hipotetikus napra, amely egy hétfői nap, DB-vel és mindegyik emberünk 18 egységet fogyaszt a kocsmából.
* Az előző input adat mellett láthatjuk a notebook alján, hogy milyen predikciók jöttek a fogyasztásra, én például egy ilyen napon elvileg mind az 5 termékből fogyasztok (ez valoszinuleg tényleg be is következne)

In [339]:
import numpy as np 
import pandas as pd 
import random
import surprise
from tqdm.notebook import tqdm
from sklearn.datasets import make_multilabel_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

In [340]:
data = pd.read_parquet("mldata_ratings_top20prod.parquet")

In [341]:
data["Név"].nunique()

10

In [458]:
data.head()

,date,Név,Mennyiség,event1,event2,event_count,weekday,termek__cappy multivitamin (dl),termek__coca-cola (dl),termek__kőbányai,termek__lays,termek__uncsi,termek__notermek,ratepred__cappy multivitamin (dl),ratepred__coca-cola (dl),ratepred__kőbányai,ratepred__lays,ratepred__uncsi,User_ID
0,2021-07-16,Barabás Dániel,0.0,Mardekár buli,Nincs esemény,1.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.900000,0.9,0.900000,0.765516,0.900000,8
1,2021-07-16,Barát Andor Kornél,3.0,Mardekár buli,Nincs esemény,1.0,4,0.0,0.0,0.0,0.0,1.0,0.0,0.900000,0.9,0.199785,0.187536,0.198922,1
2,2021-07-16,Gyenes Márk,0.0,Mardekár buli,Nincs esemény,1.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.300554,0.9,0.417971,0.900000,0.416655,7
3,2021-07-16,Herbák Mihály,2.0,Mardekár buli,Nincs esemény,1.0,4,0.0,0.0,0.0,0.0,1.0,0.0,0.458081,0.9,0.900000,0.467351,0.485573,2
4,2021-07-16,Kaló Eszter,0.0,Mardekár buli,Nincs esemény,1.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.223340,0.9,0.259773,0.244971,0.900000,10


In [343]:
név_id = pd.DataFrame(
    {
        "Név": list(data["Név"].unique()),
        "User_ID": range(1, len(data["Név"].unique()) + 1),
    }
)


In [344]:
# generate binary values using get_dummies
dum_df = pd.get_dummies(data.loc[:,"termek"], columns=["termek"], prefix="termek_" )
# merge with main df bridge_df on key values
data = data.join(dum_df)
data.shape

(15474, 22)

In [348]:
agg_dict = {
    "Mennyiség": "sum",
    "termek__cappy multivitamin (dl)": "max",
    "termek__coca-cola (dl)": "max",
    "termek__kőbányai": "max",
    "termek__lays": "max",
    "termek__uncsi": "max",
}
agg_dict2 = {
    "termek__cappy multivitamin (dl)": "max",
    "termek__coca-cola (dl)": "max",
    "termek__kőbányai": "max",
    "termek__lays": "max",
    "termek__uncsi": "max",
}

In [350]:
asd = (
    data.groupby(["Név", "date", "termek"])
    .agg(agg_dict)
    .reset_index()
    .loc[lambda _df: _df["Mennyiség"] != 0, :]
    .groupby(["Név", "date"])
    .agg(agg_dict2)
    .reset_index()
)

In [351]:
termek_col_list = list(asd.iloc[:,-10:].columns)

In [352]:
asd = asd.assign(
    termek__notermek=lambda _df: _df.loc[:, termek_col_list].sum(axis=1)
)

C:\Users\Velkey Artúr\AppData\Local\Temp\ipykernel_28236\630124592.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  termek__notermek=lambda _df: _df.loc[:, termek_col_list].sum(axis=1)


In [353]:
asd["termek__notermek"] = np.where(asd["termek__notermek"] > 0, 0,1)

In [355]:
preprocess = pd.concat(
    [
        data.groupby(["Név", "termek"]).agg({"rate_pred": "first"}).reset_index(),
        pd.get_dummies(
            data.groupby(["Név", "termek"])
            .agg({"rate_pred": "first"})
            .reset_index()
            .loc[:, "termek"],
            columns=["termek"],
            prefix="ratepred_",
        ),
    ],
    axis=1,
)

In [357]:
preprocess_cols = [
    "ratepred__cappy multivitamin (dl)",
    "ratepred__coca-cola (dl)",
    "ratepred__kőbányai",
    "ratepred__lays",
    "ratepred__uncsi",
]

In [358]:
preprocess.loc[:,preprocess_cols] = preprocess.loc[:,preprocess_cols].astype(float)

In [359]:
rate_agg_dict = {
    "ratepred__cappy multivitamin (dl)" : "sum",
    "ratepred__coca-cola (dl)" : "sum",
    "ratepred__kőbányai" : "sum",
    "ratepred__lays" : "sum",
    "ratepred__uncsi" : "sum",
}

In [360]:
preprocess["rate_pred"] = preprocess["rate_pred"].replace({np.nan : 0.9})

In [361]:
concatelni_valo_df = pd.concat(
    [
        preprocess.loc[:, ["Név", "termek"]],
        preprocess.loc[:, preprocess_cols].multiply(
            preprocess["rate_pred"], axis="index"
        ),
    ],
    axis=1,
).groupby("Név").agg(rate_agg_dict).reset_index()

In [362]:
data = (
    data.groupby(["date", "Név"])
    .agg(
        {
            "Mennyiség": "sum",
            "event1": "first",
            "event2": "first",
            "event_count": "first",
            "weekday": "first",
        }
    )
    .reset_index()
    .merge(asd, on=["Név", "date"], how="left")
    .replace({np.nan: 0})
    .merge(concatelni_valo_df, on="Név", how="left")
)

### Multilabel classification

In [363]:
data = data.merge(név_id,
    on="Név",
    how="left",
)

In [364]:
data["date"].unique()[153]

numpy.datetime64('2021-12-16T00:00:00.000000000')

In [365]:
data_train = data.loc[lambda _df: _df["date"] < pd.to_datetime("2021-12-16")]
data_test = data.loc[lambda _df: _df["date"] > pd.to_datetime("2021-12-15")].loc[
    lambda _df: _df["date"] < pd.to_datetime("2022-04-16")
]

In [372]:
labels_to_predict = [
    "termek__cappy multivitamin (dl)",
    "termek__coca-cola (dl)",
    "termek__kőbányai",
    "termek__lays",
    "termek__uncsi",
]

labels_to_predict_2 = [
    "termek__cappy multivitamin (dl)",
    "termek__coca-cola (dl)",
    "termek__dreher",
    "termek__gordons",
    "termek__kronenbourg blanc",
    "termek__kőbányai",
    "termek__lays",
    "termek__pizza",
    "termek__uncsi",
    "termek__zubrowka",
]

In [377]:
x_train_en = (
    data_train.set_index("date")
    .loc[lambda _df: _df["Név"] == "Velkey Artúr"]
    .drop(labels_to_predict, axis=1)
    .drop(["Név", "event1", "event2", "termek__notermek"], axis=1)
)
y_train_en = (
    data_train.set_index("date")
    .loc[lambda _df: _df["Név"] == "Velkey Artúr"]
    .loc[:, labels_to_predict]
)
x_test_en = (
    data_test.set_index("date")
    .loc[lambda _df: _df["Név"] == "Velkey Artúr"]
    .drop(labels_to_predict, axis=1)
    .drop(["Név", "event1", "event2", "termek__notermek"], axis=1)
)
y_test_en = (
    data_test.set_index("date")
    .loc[lambda _df: _df["Név"] == "Velkey Artúr"]
    .loc[:, labels_to_predict]
)

In [380]:
y_test_en = y_test_en.astype("category")
x_test_en = x_test_en.astype("category")

In [382]:
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())

# Training logistic regression model on train data
classifier.fit(x_train_en, y_train_en)

# predict
predictions = classifier.predict(x_test_en)

In [384]:
classification_report(
    y_test_en.to_numpy().astype(int),
    predictions.astype(int),
    output_dict=True,
    target_names=labels_to_predict,
    zero_division = 1
)

{'termek__cappy multivitamin (dl)': {'precision': 0.5,
  'recall': 0.3333333333333333,
  'f1-score': 0.4,
  'support': 3},
 'termek__coca-cola (dl)': {'precision': 0.6428571428571429,
  'recall': 0.75,
  'f1-score': 0.6923076923076924,
  'support': 12},
 'termek__kőbányai': {'precision': 1.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 12},
 'termek__lays': {'precision': 0.25,
  'recall': 0.2222222222222222,
  'f1-score': 0.23529411764705882,
  'support': 9},
 'termek__uncsi': {'precision': 1.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 3},
 'micro avg': {'precision': 0.5,
  'recall': 0.3076923076923077,
  'f1-score': 0.380952380952381,
  'support': 39},
 'macro avg': {'precision': 0.6785714285714286,
  'recall': 0.26111111111111107,
  'f1-score': 0.26552036199095025,
  'support': 39},
 'weighted avg': {'precision': 0.6785714285714286,
  'recall': 0.3076923076923077,
  'f1-score': 0.29808562478245737,
  'support': 39},
 'samples avg': {'precision': 0.9366391184573003,
  're

### Minden személyre külön kell futtatni a modellt, hiszen a személyek tök külön álló tulajdonságokkal rendelkeznek, nem érdemes egy mintaként kezelni őket

In [453]:
lista = [
    "Barabás Dániel",
    "Barát Andor Kornél",
    "Kaló Eszter",
    "Magyar Gergely",
    "Velkey Artúr",
]

In [472]:
random.seed(42)
teszt2 = (
    data.loc[lambda _df: _df["Név"].isin(lista)]
    .drop_duplicates("Név")
    .assign(
        Mennyiség=random.randint(8, 20),
        event_count=random.randint(1, 3),
        weekday=random.randint(1, 7),
    )
    .loc[
        :,
        [
            "Név",
            "Mennyiség",
            "event_count",
            "weekday",
            "ratepred__cappy multivitamin (dl)",
            "ratepred__coca-cola (dl)",
            "ratepred__kőbányai",
            "ratepred__lays",
            "ratepred__uncsi",
        ],
    ]
)

In [473]:
teszt2

,Név,Mennyiség,event_count,weekday,ratepred__cappy multivitamin (dl),ratepred__coca-cola (dl),ratepred__kőbányai,ratepred__lays,ratepred__uncsi
0,Barabás Dániel,18,1,1,0.900000,0.9,0.900000,0.765516,0.900000
1,Barát Andor Kornél,18,1,1,0.900000,0.9,0.199785,0.187536,0.198922
4,Kaló Eszter,18,1,1,0.223340,0.9,0.259773,0.244971,0.900000
6,Magyar Gergely,18,1,1,0.900000,0.9,0.900000,0.900000,0.900000
9,Velkey Artúr,18,1,1,0.673311,0.9,0.216125,0.900000,0.215210


In [474]:
date = "2022-05-05"
predicted_consumed_goods_per_person = []
for nev in tqdm(data.loc[lambda _df: _df["Név"].isin(lista), "Név"].unique()):

    to_predict = teszt2.loc[lambda _df: _df["Név"] == nev].drop("Név", axis = 1)

    x_train_en = (
        data_train.set_index("date")
        .loc[lambda _df: _df["Név"] == nev]
        .drop(labels_to_predict, axis=1)
        .drop(["Név", "event1", "event2", "termek__notermek", "User_ID"], axis=1)
    )
    y_train_en = (
        data_train.set_index("date")
        .loc[lambda _df: _df["Név"] == nev]
        .loc[:, labels_to_predict]
    )
    x_test_en = (
        data_test.set_index("date")
        .loc[lambda _df: _df["Név"] == nev]
        .drop(labels_to_predict, axis=1)
        .drop(["Név", "event1", "event2", "termek__notermek", "User_ID"], axis=1)
    )
    y_test_en = (
        data_test.set_index("date")
        .loc[lambda _df: _df["Név"] == nev]
        .loc[:, labels_to_predict]
    )

    # initialize classifier chains multi-label classifier
    classifier = ClassifierChain(LogisticRegression())

    # Training logistic regression model on train data
    classifier.fit(x_train_en, y_train_en)

    # predict
    # predictions = classifier.predict(x_test_en)

    datashit = (
        pd.concat(
            [
                y_test_en.reset_index().loc[:, "date"],
                pd.DataFrame(classifier.predict(to_predict)).rename(
                    columns=col_replace_dict
                ),
            ],
            axis=1,
        )
        .assign(Név=nev)
        .dropna()
    )
    datashit["date"] = date
    predicted_consumed_goods_per_person.append(datashit)

  0%|          | 0/5 [00:00<?, ?it/s]

In [475]:
pd.concat(predicted_consumed_goods_per_person)

,date,termek__cappy multivitamin (dl),termek__coca-cola (dl),termek__kőbányai,termek__lays,termek__uncsi,Név
0,2022-05-05,1.0,1.0,1.0,1.0,1.0,Barabás Dániel
0,2022-05-05,0.0,1.0,0.0,0.0,0.0,Barát Andor Kornél
0,2022-05-05,0.0,1.0,1.0,0.0,1.0,Kaló Eszter
0,2022-05-05,1.0,1.0,1.0,1.0,1.0,Magyar Gergely
0,2022-05-05,1.0,1.0,1.0,1.0,1.0,Velkey Artúr
